In [ ]:
import pandas as pd
from groq import Groq
import time

client_groq = Groq(
    api_key="xxxx"
)

In [ ]:
df = pd.read_excel('../Spain_train_set_2019.xlsx')

In [ ]:
def clasifica_groq(tweet):
    prompt =  f"""Eres un modelo clasificador de textos. 
    Tienes que responder si el siguiente texto hace referencia a "POS" (Positivo), "NEG" (Negativo) o "NEU" (Neutral).  
    **Limítate a devolver solo la categoría (POS, NEG, NEU), nada más.**
    
    Texto: "{tweet}"
    """
    try:
        response = client_groq.chat.completions.create(
            messages = [ 
                { "role": "user", "content": prompt }
            ],
            model="llama-3.3-70b-versatile",
            temperature=0, 
        )
        return response.choices[0].message.content.strip()
        time.sleep(2)
    except Exception as e:
        print(e)

In [ ]:
df["Polarity_llama-3.3-70b-versatile_ZeroShot"] = df["Content"].apply(clasifica_groq)
df

,ID,Content,Lang,Polarity,Polarity_llama-3.1-8b-instant_ZeroShot
0,768213567418036224,@myendlesshazza a. que puto mal escribo b. me ...,es,NEG,NEG
1,768221670255493120,Quiero mogollón a @AlbaBenito99 pero sobretodo...,es,POS,POS
2,768221021300264964,Vale he visto la tia bebiendose su regla y me ...,es,NEG,NEG
3,768220253730009091,@Yulian_Poe @guillermoterry1 Ah. mucho más por...,es,POS,NEU
4,768231229439311872,@toNi_end seria mejor que dejasen de emitir es...,es,NEG,NEG
...,...,...,...,...,...
963,814846333601320960,Para mí mi mejor año fue el 2015,es,POS,POS
964,813731371076243461,Hoy va a ser un dia muy largo...,es,NEG,NEG
965,818399956792905728,11:11 que no me encuentre mal,es,NEU,NEG
966,815715581878009858,Es que son muy bonitas jo,es,POS,POS


In [ ]:
def clasifica_groq2(tweet):
    prompt =  f"""Eres un experto en análisis de sentimiento de textos en español. 
    Tu tarea es leer el siguiente tweet y clasificarlo en una de estas categorías:  
    POS → Expresa felicidad, satisfacción o una emoción positiva.  
    NEG → Contiene quejas, frustración o emociones negativas.  
    NEU → No tiene emoción clara o es informativo, o relamente no dice nada. 
    
    Ejemplo:
    Texto: "Hoy es un día maravilloso" Respuesta: POS
    Texto: "No me gusta el frío" Respuesta: NEG
    Texto: "El coche es azul" Respuesta: NEU   
    **Limítate a devolver solo la categoría, nada más.**
    
    Texto: "{tweet}"
    """
    try:
        response = client_groq.chat.completions.create(
            messages = [ 
                { "role": "user", "content": prompt }
            ],
            model="llama-3.3-70b-versatile",
            temperature=0,
        )
        return response.choices[0].message.content.strip()
        time.sleep(2)
    except Exception as e:
        print(e)

In [ ]:
df["Polarity_llama-3.3-70b-versatile_FewShot"] = df["Content"].apply(clasifica_groq2)
df

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11680\3896089981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin["Polarity_llama-3.3-70b-versatile_FewShot"] = df_fin["Content"].apply(clasifica_groq2)


In [ ]:
def clasifica_groq3(tweet):
    prompt =  f"""Eres un experto en análisis de sentimiento de textos en español.
    Clasifica la siguiente bateria de tweets en una de las siguientes categorías:  
    POS → Si expresa alegría, emoción positiva o satisfacción.  
    NEG → Si muestra quejas, descontento o emociones negativas.  
    NEU → Si es un comentario informativo sin carga emocional, o relamente no dice nada.
    Hazlo Paso a paso siguiendo el siguiente formato:
    
    1. Identificación de palabras clave: [Enumera palabras o frases clave en el texto] 
    2. Interpretación del tono general: [Explica si el tono es positivo, negativo o neutro basado en el contexto]  
    3. Detección de matices: [Considera si hay ironía, emoción oculta o lenguaje ambiguo]
    4. Polaridad final (Positivo, Negativo, neutral): [POS / NEG / NEU]
    
    No queiro que añadas más de lo descrito, limitate a responder lo que se te pide y como se te pide.
    
    Ejemplo:
    Texto: "No está mal, pero esperaba algo mucho mejor."
    1. Identificación de palabras clave: "No está mal", "esperaba algo mucho mejor"
    2. Interpretación del tono general: El texto no es completamente negativo, pero expresa una decepción.
    3. Detección de matices: Aunque "No está mal" podría parecer positivo, la segunda parte del texto implica insatisfacción.
    4. Polaridad final: NEG
    
    Texto: "{tweet}"
    """
    try:
        response = client_groq.chat.completions.create(
            messages = [ 
                { "role": "user", "content": prompt }
            ],
            model="llama-3.3-70b-versatile",
            temperature=0, 
        )
        result = response.choices[0].message.content.strip()
        time.sleep(2)
        if "4. Polaridad final:" in result:
            parts = result.split("4. Polaridad final:")
            categoria = parts[1].strip()
            crudo = parts[0].strip()
            if "\n" in categoria:
                categoria = categoria.split("\n")[0]
            return categoria, crudo
    except Exception as e:
        print(e)

In [22]:
df[["Polarity_llama-3.3-70b-versatile_Change_Of_Thought", "Raw_llama-3.3-70b-versatile_Change_Of_Thought"]] = df["Content"].apply(lambda tweet: pd.Series(clasifica_groq3(tweet)))
df

,ID,Content,Lang,Polarity,Polarity_llama-3.3-70b-versatile_Change_Of_Thought,Raw_llama-3.3-70b-versatile_Change_Of_Thought
179,767764798738206721,Ya a saber cuando vuelves a hacer un #FollowEx...,es,NEG,NEG,"1. Identificación de palabras clave: ""lloro"", ..."
180,767764285351202817,¿Algún día seré feliz por tener tu follow? @Bl...,es,NEG,POS,"1. Identificación de palabras clave: ""¿Algún d..."
181,767754109915131904,@CiudadanO_O @Moralzarzal con lo fácil q sería...,es,NEG,NEG,"1. Identificación de palabras clave: ""qué pena..."
182,767752844258652160,Hoy microaventura en #kayak con dos expertos k...,es,POS,POS,"1. Identificación de palabras clave: ""microave..."
183,768047633613086721,Quiero esta figura de Madison ojalá que la pr...,es,POS,POS,"1. Identificación de palabras clave: ""Quiero"",..."
184,768046274717347840,@ladyfrigopie no lo has dicho. Eres rara.. bue...,es,NEG,NEU,"1. Identificación de palabras clave: ""no lo ha..."


In [ ]:
df.to_excel('Clasificación_llama-3.3-70b-versatile.xlsx', index=False)